17 características clínicas para predizer o estado de sobrevida de pacientes com cirrose hepática. Os estados de sobrevida incluem 0 = D (óbito), 1 = C (censurado), 2 = CL (censurado devido ao transplante hepático).

Os dados fornecidos são provenientes de um estudo da Mayo Clinic sobre cirrose biliar primária (CBP) do fígado realizado de 1974 a 1984.

Informações adicionais sobre as variáveis
1. ID: identificador único
2. N_Days: número de dias entre o registro e o início do óbito, transplante ou tempo de análise do estudo em julho de 1986
3. Status: estado do paciente C (censurado), CL (censurado devido a tx hepática) ou D (óbito)
4. Droga: tipo de droga D-penicilamina ou placebo
5. Idade: idade em [dias]
6. Sexo: M (macho) ou F (fêmea)
7. Ascite: presença de ascite N (Não) ou Y (Sim)
8. Hepatomegalia: presença de hepatomegalia N (Não) ou Y (Sim)
9. Aranhas: presença de aranhas N (Não) ou Y (Sim)
10. Edema: presença de edema N (sem edema e sem terapia diurética para edema), S (edema presente sem diuréticos ou edema resolvido por diuréticos) ou Y (edema apesar da terapia diurética)
11. Bilirrubina: bilirrubina sérica em [mg/dl]
12. Colesterol: colesterol sérico em [mg/dl]
13. Albumina: albumina em [gm/dl]
14. Cobre: cobre urinário em [ug/dia]
15. Alk_Phos: fosfatase alcalina em [U/litro]
16. SGOT: SGOT em [U/ml]
17. Triglicerídeos: triglicerídeos em [mg/dl]
18. Plaquetas: plaquetas por cúbico [ml/1000]
19. Protrombina: tempo de protrombina em segundos [s]
20. Estádio: estádio histológico da doença (1, 2, 3 ou 4)

Rótulos de classe
Status: estado do doente 0 = D (óbito), 1 = C (censurado), 2 = CL (censurado devido a transplante hepático)

In [100]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.neural_network import MLPClassifier
import numpy as np

1. Carregamento do conjunto de dados

In [2]:
df = pd.read_csv('cirrhosis.csv')
df.head()

,ID,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,1,400,D,D-penicillamine,21464,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,2,4500,C,D-penicillamine,20617,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,3,1012,D,D-penicillamine,25594,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,4,1925,D,D-penicillamine,19994,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
4,5,1504,CL,Placebo,13918,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0


2. Informação sobre os dados

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             418 non-null    int64  
 1   N_Days         418 non-null    int64  
 2   Status         418 non-null    object 
 3   Drug           312 non-null    object 
 4   Age            418 non-null    int64  
 5   Sex            418 non-null    object 
 6   Ascites        312 non-null    object 
 7   Hepatomegaly   312 non-null    object 
 8   Spiders        312 non-null    object 
 9   Edema          418 non-null    object 
 10  Bilirubin      418 non-null    float64
 11  Cholesterol    284 non-null    float64
 12  Albumin        418 non-null    float64
 13  Copper         310 non-null    float64
 14  Alk_Phos       312 non-null    float64
 15  SGOT           312 non-null    float64
 16  Tryglicerides  282 non-null    float64
 17  Platelets      407 non-null    float64
 18  Prothrombi

3. Tratamento dos dados

3.1 Age (idade) está em dias, vamos converter para anos para ajudar a reduzir a escala dos dados

In [4]:
# Criando uma nova coluna com idade em anos (AgeYears) e adicionando no dataframe
df['AgeYears'] = df['Age'] / 365.25

# Reorganizar as colunas para colocar 'AgeYears' antes de Stage
cols = list(df.columns)
cols.remove('AgeYears')
cols.insert(cols.index('Stage'), 'AgeYears')

# Reorganizar o dataframe de acordo com a nova ordem das coluns
df = df[cols]

# Removendo a coluna Age (idade em dias)
df.drop(columns=['Age'], inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID             418 non-null    int64  
 1   N_Days         418 non-null    int64  
 2   Status         418 non-null    object 
 3   Drug           312 non-null    object 
 4   Sex            418 non-null    object 
 5   Ascites        312 non-null    object 
 6   Hepatomegaly   312 non-null    object 
 7   Spiders        312 non-null    object 
 8   Edema          418 non-null    object 
 9   Bilirubin      418 non-null    float64
 10  Cholesterol    284 non-null    float64
 11  Albumin        418 non-null    float64
 12  Copper         310 non-null    float64
 13  Alk_Phos       312 non-null    float64
 14  SGOT           312 non-null    float64
 15  Tryglicerides  282 non-null    float64
 16  Platelets      407 non-null    float64
 17  Prothrombin    416 non-null    float64
 18  AgeYears  

In [5]:
# Resumo estatístico de AgeYears
df['AgeYears'].describe()

count    418.000000
mean      50.741551
std       10.447214
min       26.277892
25%       42.832307
50%       51.000684
75%       58.240931
max       78.439425
Name: AgeYears, dtype: float64

3.2 Verificar se há dados ausentes

In [6]:
df.isna().sum()

ID                 0
N_Days             0
Status             0
Drug             106
Sex                0
Ascites          106
Hepatomegaly     106
Spiders          106
Edema              0
Bilirubin          0
Cholesterol      134
Albumin            0
Copper           108
Alk_Phos         106
SGOT             106
Tryglicerides    136
Platelets         11
Prothrombin        2
AgeYears           0
Stage              6
dtype: int64

Podemos notar que há alguns dados faltantes em algumas colunas.

Segundo a orientação dos desenvolvedores desse conjunto de dados, os valores ausente em Drug devem ser removidos e as  colunas numéricas com valores ausentes devem ser preenchidos com a respectiva média. As colunas categóricas devem ser preenchidas com o valor mais frequente (moda)

In [7]:
# Removendo todas as linhas de Drug com valores faltantes
df.dropna(subset=['Drug'], inplace=True)

# Verificando se ainda há dados faltantes na coluna
df['Drug'].isna().sum()

0

In [8]:
# Subsituir com a respectiva média os valores faltantes para as colunas numéricas
cols_numericas = ['Cholesterol', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin', 'Stage']

for col in cols_numericas:
    mean_value = df[col].mean()
    df[col].fillna(mean_value, inplace=True)
    
# Para preencher NaNs nas colunas categóricas com a moda:
cols_categoricas = ['Ascites', 'Hepatomegaly', 'Spiders']

for col in cols_categoricas:
    mode_value = df[col].mode()[0]
    df[col].fillna(mode_value, inplace=True)

In [9]:
# Verificando se há dados ausentes ainda
df.isna().sum()

ID               0
N_Days           0
Status           0
Drug             0
Sex              0
Ascites          0
Hepatomegaly     0
Spiders          0
Edema            0
Bilirubin        0
Cholesterol      0
Albumin          0
Copper           0
Alk_Phos         0
SGOT             0
Tryglicerides    0
Platelets        0
Prothrombin      0
AgeYears         0
Stage            0
dtype: int64

3.3 Codificar colunas categóricas (transformar para numérica) com one-hot-encoding

Colunas categóricas:

Status 

Drug   

Sex     

Ascites    

Hepatomegaly   

Spiders            

Edema          

In [10]:
df.columns

Index(['ID', 'N_Days', 'Status', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly',
       'Spiders', 'Edema', 'Bilirubin', 'Cholesterol', 'Albumin', 'Copper',
       'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin',
       'AgeYears', 'Stage'],
      dtype='object')

In [11]:
# Aplicação do One-Hot
cols_to_encode = ['Status', 'Drug', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema']

# Criar um novo DataFrame com as colunas categóricas originais
df_categorical = df[cols_to_encode]

# Aplicação do One-Hot Encoding apenas nas colunas categóricas originais
df_encoded = pd.get_dummies(df_categorical,columns=cols_to_encode)
        
#Reinicar o índice do Dataframe df_encoded para evitar conflitos
df_encoded.reset_index(drop=True, inplace=True)

# Manter apenas as colunas numéricas do DataFrame original
df_numeric = df.drop(columns=cols_to_encode)
 
# Concatenar os DataFrames ao longo das colunas
df_concat = pd.concat([df_numeric, df_encoded], axis=1)

df_concat.head()

,ID,N_Days,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,...,Sex_M,Ascites_N,Ascites_Y,Hepatomegaly_N,Hepatomegaly_Y,Spiders_N,Spiders_Y,Edema_N,Edema_S,Edema_Y
0,1,400,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,...,False,False,True,False,True,False,True,False,False,True
1,2,4500,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,...,False,True,False,False,True,False,True,True,False,False
2,3,1012,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,...,True,True,False,True,False,True,False,False,True,False
3,4,1925,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,...,False,True,False,False,True,False,True,False,True,False
4,5,1504,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,...,False,True,False,False,True,False,True,True,False,False


4. Definindo as variáveis preditoras (X) e a alvo (y)

In [12]:
X = df.drop(columns=['ID', 'Status'])
y = df['Status']

5. Separação em treino e teste

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# <h4>REGRESSÃO LOGÍSTICA</h4>

1. Criar e treinar o modelo

In [14]:
# Antes temos que codificar as variáveis categóricas em números
X_train = X_train.select_dtypes(include=[np.number])
X_test = X_test.select_dtypes(include=[np.number])

In [15]:
reg_log = LogisticRegression()
reg_log.fit(X_train, y_train)

c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

2. Previsão com o modelo

In [16]:
reg_log_pred = reg_log.predict(X_test)

3. Avaliação do modelo

In [17]:
print('Relatório de Classificação:\n')
report = classification_report(y_test, reg_log_pred)
print(f'{report}')

Relatório de Classificação:

              precision    recall  f1-score   support

           C       0.69      0.86      0.77        29
          CL       0.00      0.00      0.00         8
           D       0.74      0.77      0.75        26

    accuracy                           0.71        63
   macro avg       0.48      0.54      0.51        63
weighted avg       0.63      0.71      0.67        63



c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

# <h4>RANDOM FOREST</h4>

1. Criar e treinar o modelo

In [62]:
rnd_forest = RandomForestClassifier(n_estimators=250, max_depth=5)
rnd_forest.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, n_estimators=250)

2. Previsões

In [58]:
rnd_forest_pred = rnd_forest.predict(X_test)

3. Avaliação do modelo

In [59]:
print('Relatório de Classificação:\n')
report = classification_report(y_test, rnd_forest_pred)
print(f'{report}')

Relatório de Classificação:

              precision    recall  f1-score   support

           C       0.64      0.79      0.71        29
          CL       0.00      0.00      0.00         8
           D       0.63      0.65      0.64        26

    accuracy                           0.63        63
   macro avg       0.42      0.48      0.45        63
weighted avg       0.55      0.63      0.59        63



c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\marce\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

No geral, o modelo random forest parece ter um bom desempenho na classificação das classes C e D, mas está tendo dificuldades significativas com a classe CL.

É importante considerar o desequilíbrio de classes, pois isso pode afetar o desempenho do modelo. Vamos então ajustar o modelo, reequilibrar o conjunto de dados ver se  melhora o desempenho da classe CL.

In [65]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

In [81]:

y = df_concat['Status_CL']

#df_concat.drop(columns=['Status'], inplace=True)

rnd_forest = RandomForestClassifier(n_estimators=150, max_depth=3)

n_splits = 5
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

accuracy_scores = []

for train_index, test_index in stratified_kfold.split(X, y):
    X_train, X_test = df_concat.iloc[train_index], df_concat.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    rnd_forest.fit(X_train, y_train)
    rnd_forest_pred = rnd_forest.predict(X_test)
    
    accuracy = accuracy_score(y_test, rnd_forest_pred)
    accuracy_scores.append(accuracy)
    
mean_accuracy = sum(accuracy_scores) / n_splits

print(f'Média da acurácia: {mean_accuracy}')

Média da acurácia: 0.9840245775729647


Agora podemos notar que o desempenho melhorou muito, de 0.63 foi para 0.98

# <h4>REDE NEURAL</h4>

In [104]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 100, 100), activation="relu", solver="adam")
mlp.fit(X_train, y_train)

# Fazer previsões com o modelo de rede neural
y_pred_mlp = mlp.predict(X_test)

# Calcular a acurácia do modelo de rede neural
accuracy_mlp = accuracy_score(y_test, y_pred_mlp)

print("Acurácia do modelo de rede neural:", accuracy_mlp)

Acurácia do modelo de rede neural: 0.9354838709677419
